In [1]:
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipyevents import Event
from IPython.display import clear_output
from matplotlib.patches import Patch
import warnings

warnings.filterwarnings('ignore')

In [2]:
df_ubications = pd.read_excel("metadata.xlsx")
df_ica = pd.read_excel("DatosICA.xlsx", sheet_name="ICA_texto")

df_ica['Fecha'] = pd.to_datetime(df_ica['Fecha'])

In [3]:
def coordsToDecimal(coord):
    "Change Coordinates to Decimal"
    coord = coord.replace("′", "'").replace("″", '"')
    deg, minutes, seconds, direction =  re.split('[°\'"]', coord)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

df_ubications["Longitud"] = df_ubications["Longitud"].apply(coordsToDecimal)
df_ubications["Latitud"] = df_ubications["Latitud"].apply(coordsToDecimal)

In [4]:
provincias = pd.read_csv("provincias-espanolas.csv", sep=";")[["Provincia", "Geo Shape"]]
provincias["Geo Shape"] = provincias["Geo Shape"].apply(
    lambda x: json.loads(x)["coordinates"]
)

In [5]:
class Marker:
    """
    Clase que simboliza los marcadores en el mapa. 
    Sus parámetros son coordenada x, coordenada y, y tipo de estación.
    """
    def __init__(self, nombre, x, y, ica="Sin datos", color=None):
        self.x = x
        self.y = y
        self.ica = ica
        self.nombre = nombre
        self.color = color
    
    def colour(self):
        """
        Función que devuelve el color correspondiente al ICA.
        Se ejecuta automaticamente al iniciar la clase
        """
        
        colorCode = {
            "Buena": "cyan",
            "Razonablemente buena": "green",
            "Regular": "yellow",
            "Desfavorable": "red",
            "Muy desfavorable": "garnet",
            "Extremadamente desfavorable": "purple",
            "Dato no válido": "grey"   
        }
        
        return colorCode[self.ica]

In [6]:
print(provincias["Geo Shape"].head())

# Si nos fijamos, hay si hay más de un polígono en una misma provincia, el array tendrá profundidad 4,
# mientras que si solo tiene un polígono, la profundidad será 3.
# Por ello, se ha realizado un manejo de condicionales y excepciones para poder mostrar todas las provincias.

0    [[[-5.01385768137012, 41.16424852689012], [-5....
1    [[[[-5.437333022160619, 36.064233108628436], [...
2    [[[[-0.987141629168891, 39.980724854021794], [...
3    [[[-2.766827131575509, 38.53296916076714], [-2...
4    [[[[-4.254107365433731, 40.65866522884571], [-...
Name: Geo Shape, dtype: object


In [19]:
df_ica["Fecha"] = pd.to_datetime(df_ica["Fecha"])
df_ica["Fecha"] = df_ica["Fecha"].dt.date

In [21]:
def generarMapa():
    # Selección Fecha
    global fecha_seleccionada
    fecha_seleccionada = desplegable.value
    fila = df_ica.loc[df_ica['Fecha'] == fecha_seleccionada]
    
    # Da valores de color e ICA
    for index, row in fila.iterrows():
        n = len(row[1:])
        for i in range(1, n+1):
            markers_list[i-1].ica = row[i]
            markers_list[i-1].color = markers_list[i-1].colour()

    # Genera una figura
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

    # Genera el mapa
    for index, lugar in provincias.iterrows():
        place = lugar[0]
        borde = lugar[1]

        if len(borde) > 1:
            for i in range(len(borde)):
                a = np.array(borde[i][0])
                try:
                    if place == "Las Palmas" or place == "Santa Cruz de Tenerife": 
                        ax1.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )
                    else:
                        ax2.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )
                except Exception as e:
                    a = np.array(borde[i])
                    if place == "Las Palmas" or place == "Santa Cruz de Tenerife": 
                        ax1.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )
                    else:
                        ax2.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )
        else:
            a = np.array(borde[0])
            try:
                if place == "Las Palmas" or place == "Santa Cruz de Tenerife": 
                        ax1.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )
                else:
                        ax2.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )
            except Exception as e:
                if place == "Las Palmas" or place == "Santa Cruz de Tenerife": 
                        ax1.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )
                else:
                        ax2.plot(
                            a[:,0],
                            a[:,1],
                            "lightgrey",
                            markersize=0.1
                        )

    # Genera los marcadores
    for lugar in markers_list:
        if lugar.nombre == "Tenerife":
            ax1.plot(lugar.x, lugar.y, color=lugar.color, marker="o", markersize=10, linewidth=3, markeredgecolor='black', label=lugar.nombre)
        else:
            ax2.plot(lugar.x, lugar.y, color=lugar.color, marker="o", markersize=10, linewidth=3, markeredgecolor='black', label=lugar.nombre)

    # Ajustar límites        
    ax1.set_xlim(-19, -13)
    ax1.set_ylim(27, 30)
    ax1.set_title('Islas Canarias')
    ax1.axis('off')

    ax2.set_xlim(-10, 5)
    ax2.set_ylim(35, 45)
    ax2.set_title('Península')
    ax2.axis('off')

    # Separador
    plt.axvline(x=-10, color='black', linestyle='-')
    plt.subplots_adjust(hspace=0.4)

    # Leyenda
    colores = ["cyan", "green", "yellow", "red", "#6B0304", "purple", "grey"]
    labels = ["Buena", "Razonablemente buena", "Regular", "Desfavorable", "Muy desfavorable", "Extremadamente desfavorable", "Sin datos"]
    parches = [Patch(color=color) for color in colores]

    # Mostrar imagen y leyenda
    ax1.legend(parches, labels, loc='lower left')
    plt.show()
    
# Generar resumen de las estaciones
def generar_resumen():
    dic_resumen = dict()
    for lugar in markers_list:
        if lugar.ica in dic_resumen:
            dic_resumen[lugar.ica].append(lugar.nombre)
        else:
            dic_resumen[lugar.ica] = [lugar.nombre]
            
    global fecha_seleccionada
    fecha_seleccionada = str(fecha_seleccionada)[:11]
    print("# ------------------------------------------ #")
    print(f"       Generando Resumen : {fecha_seleccionada}")
    print("# ------------------------------------------ # \n")

    for index, lugares in dic_resumen.items():
        print(f"#{index}:")
        for lugar in lugares:
            print(lugar)
        print("\n")
    
# Lista con los marcadores y sus posiciones y nombres
markers_list = list()
for index, row in df_ubications.iterrows():
    markers_list.append(Marker(row[2], row[4], row[3]))
    
# Función ejecutada al cambiar desplegable
def seleccionar_valor(valor):
    with out:
        clear_output()
        generarMapa()
        generar_resumen()

out = widgets.Output()

# Función desplegable
fechas = df_ica['Fecha']

desplegable = widgets.Dropdown(options=fechas, description='Fecha:')
desplegable.observe(lambda cambio: seleccionar_valor(cambio['new']), names='value')
display(desplegable, out)

Dropdown(description='Fecha:', options=(datetime.date(2009, 1, 1), datetime.date(2009, 1, 2), datetime.date(20…

Output()